In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Noise

In [28]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import models

In [24]:
# ================== CONFIG =====================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
df = pd.read_csv('/content/drive/MyDrive/AML-PROJECT/623final_all.csv')
df['ID'] = df['label'].str.extract(r'(\d+)', expand=False)
df['encoded_label'] = LabelEncoder().fit_transform(df['label'])
NUM_CLASSES = df['encoded_label'].nunique()

train_val_df, test_df = train_test_split(df, test_size=0.2, stratify=df['encoded_label'], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, stratify=train_val_df['encoded_label'], random_state=42)  # 60/20/20 split

# ================== Dataset =====================
class IrisDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['image_path']).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(row['encoded_label'], dtype=torch.long)
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
train_loader = DataLoader(IrisDataset(train_df, transform), batch_size=32, shuffle=True)
val_loader = DataLoader(IrisDataset(val_df, transform), batch_size=32, shuffle=False)
test_loader = DataLoader(IrisDataset(test_df, transform), batch_size=32, shuffle=False)

In [25]:
# ================== Autoencoder =====================
class DenseNetEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        self.encoder = densenet.features

    def forward(self, x):
        x = self.encoder(x)
        return x

class Decoder(nn.Module):
    def __init__(self, in_channels=1024):
        super().__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels, 512, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, 2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(x)

class AEClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.encoder = DenseNetEncoder()
        self.decoder = Decoder()
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.classifier = nn.Linear(1024, num_classes)

    def forward(self, x):
        feat = self.encoder(x)
        recon = self.decoder(feat)
        pooled = self.pool(feat).view(x.size(0), -1)
        out = self.classifier(pooled)
        return out, recon

In [26]:
# ================== Loss Functions =====================
loss_functions = {
    'CrossEntropyLoss': nn.CrossEntropyLoss(),
    'LabelSmoothingLoss': nn.CrossEntropyLoss(label_smoothing=0.1),
    'FocalLoss': nn.CrossEntropyLoss(),
}

optimizers = {
    'SGD': lambda params: torch.optim.SGD(params, lr=0.01, momentum=0.9),
    'AdamW': lambda params: torch.optim.AdamW(params, lr=1e-4),
}

In [29]:
# ================== Training =====================
summary = []

for loss_name, loss_fn in loss_functions.items():
    for opt_name, opt_fn in optimizers.items():
        model = AEClassifier(NUM_CLASSES).to(device)
        optimizer = opt_fn(model.parameters())

        for epoch in range(10):
            model.train()
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                preds, recon = model(images)
                cls_loss = loss_fn(preds, labels)
                recon_loss = F.mse_loss(recon, images)
                loss = cls_loss + 0.5 * recon_loss
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                preds, _ = model(images)
                all_preds.extend(torch.argmax(preds, dim=1).cpu())
                all_labels.extend(labels)

        acc = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='macro')
        prec = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        rec = recall_score(all_labels, all_preds, average='macro')
        summary.append({
            'loss_fn': loss_name,
            'optimizer': opt_name,
            'accuracy': acc,
            'f1': f1,
            'precision': prec,
            'recall': rec,
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 136MB/s]


In [30]:
# ================== Summary =====================
print("=== SUMMARY RESULTS ===")
for i, row in enumerate(summary):
    print(f"{i:2d} {row['loss_fn']:20} {row['optimizer']:8} "
          f"{row['accuracy']:.4f} {row['f1']:.4f} "
          f"{row['precision']:.4f} {row['recall']:.4f}")

=== SUMMARY RESULTS ===
 0 CrossEntropyLoss     SGD      0.8650 0.8537 0.8887 0.8650
 1 CrossEntropyLoss     AdamW    0.5875 0.5493 0.5701 0.5875
 2 LabelSmoothingLoss   SGD      0.8425 0.8269 0.8524 0.8425
 3 LabelSmoothingLoss   AdamW    0.5900 0.5597 0.5891 0.5900
 4 FocalLoss            SGD      0.8600 0.8520 0.8883 0.8600
 5 FocalLoss            AdamW    0.5450 0.5128 0.5398 0.5450


In [34]:
save_dir = "/content/drive/My Drive/AML-PROJECT/cy_auto"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

torch.save(model, os.path.join(save_dir, "aeclassifier_model_full.pkl"))
print(f"✅ successful saved: {os.path.join(save_dir, 'aeclassifier_model_full.pkl')}")

✅ 模型已保存到: /content/drive/My Drive/AML-PROJECT/cy_auto/aeclassifier_model_full.pkl
